In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from numpy import isnan
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import preprocessing
from scipy.stats import kurtosis, skew
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Ridge
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
from scipy import stats  
from scipy import interp
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc, r2_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score, cross_val_predict, cross_validate, RepeatedStratifiedKFold
from features import Random_forest, CreateFeature, sub_divide_hemispheres, loadrawdata_R, loadrawdata_U, strm_stats, extract_value_H_P, length_tract
from features import Clustering_streamlines, mahal_features, pca_correction, cal_mahal_profile, check_data
from features import evaluate_model, classify_hemispheres, load_TMS_DATA, extract_stat_per_hemis, extract_stat_dif_of_hemis, my_SVM_classifier, my_NN_classifier, stratified_cross_validation, loadrawdata_R_new
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve

In [2]:
scaler = StandardScaler()
xls = pd.ExcelFile('/Users/boshra/Desktop/final_table.xlsx')
data = xls.parse(xls.sheet_names[0])

subjN = 116
remove = np.array([5,14,21,39,42,58,90,91,93,101])

age = np.array(data['age'])
path_side = np.array(data['pathologic_side'])
path_loc = np.array(data['pathology_location'])
motor_status_preop = np.array(data['motor_status'])
motor_status_preop[np.argwhere(motor_status_preop==0)]=1
motor_status_preop[np.argwhere(motor_status_preop==4)]=1
motor_status_preop[np.argwhere(motor_status_preop==5)]=0
motor_status_preop[np.argwhere(motor_status_preop==3)]=1
motor_status_preop[np.argwhere(motor_status_preop==2)]=1
y = motor_status_preop
pathology = np.array(data['Pathology'])

gender = np.array(data['gender'])
gender[np.argwhere(gender=='m')]=1
gender[np.argwhere(gender=='f')]=0

MT_P = np.array(data['MT_P'])
MT_H = np.array(data['MT_H'])
RMT_P = np.array(data['RMT_P'])
RMT_H = np.array(data['RMT_H'])
y[71] = 0
y[81] = 1

In [4]:
# Load data related to both hemispheres
adc_data = loadrawdata_R_new('adc',subjN)
fa_data = loadrawdata_R_new('fa',subjN)
fd_data = loadrawdata_R_new('fd',subjN)
rd_data = loadrawdata_R_new('rd',subjN)
ad_data = loadrawdata_R_new('ad',subjN)
adc_data_P,fa_data_P,fd_data_P, ad_data_P, rd_data_P, adc_data_H,fa_data_H,fd_data_H, ad_data_H, rd_data_H = extract_value_H_P(adc_data, fa_data, fd_data,rd_data, ad_data, path_side, subjN)


In [5]:
#************************* Common way of getting the profile[:,1:99,:]
strm_median_P_n = np.median(fa_data_P, axis=0).T; #strm_median_P = np.delete(strm_median_P, status_remove, 0)
strm_median_H_n = np.median(fa_data_H, axis=0).T; #strm_median_H= np.delete(strm_median_H, status_remove, 0) [:,:,0:65]
strm_mean_P_fd_n = np.median(fd_data_P, axis=0).T; #strm_mean_P_fd = np.delete(strm_mean_P_fd, status_remove, 0)
strm_mean_H_fd_n = np.median(fd_data_H, axis=0).T; #strm_mean_H_fd = np.delete(strm_mean_H_fd, status_remove, 0)
strm_median_P_adc_n = np.median(adc_data_P, axis=0).T; #strm_median_P_adc = np.delete(strm_median_P_adc, status_remove, 0)
strm_median_H_adc_n = np.median(adc_data_H, axis=0).T; #strm_median_H_adc = np.delete(strm_median_H_adc, status_remove, 0)
strm_median_P_rd_n = np.median(rd_data_P, axis=0).T; #strm_median_P_adc = np.delete(strm_median_P_adc, status_remove, 0)
strm_median_H_rd_n = np.median(rd_data_H, axis=0).T;
strm_median_P_ad_n = np.median(ad_data_P, axis=0).T; #strm_median_P_adc = np.delete(strm_median_P_adc, status_remove, 0)
strm_median_H_ad_n = np.median(ad_data_H, axis=0).T;
strm = [strm_median_P_n,strm_median_H_n,strm_mean_P_fd_n,strm_mean_H_fd_n,strm_median_P_adc_n,
        strm_median_H_adc_n,strm_median_P_rd_n,strm_median_H_rd_n,strm_median_P_ad_n,strm_median_H_ad_n]
strm_str = ['strm_median_P','strm_median_H','strm_mean_P_fd','strm_mean_H_fd','strm_median_P_adc',
        'strm_median_H_adc','strm_median_P_rd','strm_median_H_rd','strm_median_P_ad','strm_median_H_ad']

In [19]:
print(strm_median_P.shape)

(116, 100)


In [6]:
#**********************************
def extract_stat_per_hemis(strm_median_P,strm_median_H):
    # stats for pataloghical hemispheres
    tmp1 = np.mean(strm_median_P, axis=1)
    tmp2 = np.std(strm_median_P, axis=1)
    tmp3 = kurtosis(strm_median_P, axis = 1, fisher = True, bias = True, nan_policy = 'propagate')
    tmp4 = skew(strm_median_P, axis = 1, bias=True)
    tmp5 = np.median(strm_median_P, axis=1)
    # stats for pataloghical hemispheres
    tmp11 = np.mean(strm_median_H, axis=1)
    tmp22 = np.std(strm_median_H, axis=1)
    tmp33 = kurtosis(strm_median_H, axis = 1, fisher = True, bias = True, nan_policy = 'propagate')
    tmp44 = skew(strm_median_H, axis = 1, bias=True)
    tmp55 = np.median(strm_median_H, axis=1)

    #featureO = np.vstack((tmp1,tmp11,tmp2,tmp22,tmp3,tmp33,tmp4,tmp44)).T    
    featureO = np.vstack((tmp1,tmp2,tmp3,tmp4)).T   
    
    feature_CST_n = (strm_median_P-strm_median_H)/(strm_median_P+strm_median_H)
    feature_CST_ds = (strm_median_P-strm_median_H)
    
    tmp1_n = np.mean(feature_CST_n, axis=1)
    tmp2_n = np.std(feature_CST_n, axis=1)
    tmp3_n = kurtosis(feature_CST_n, axis = 1, fisher = True, bias = True, nan_policy = 'propagate')
    tmp4_n = skew(feature_CST_n, axis = 1, bias=True)
    tmp1_ds = np.mean(feature_CST_ds, axis=1)
    tmp2_ds = np.std(feature_CST_ds, axis=1)
    tmp3_ds = kurtosis(feature_CST_ds, axis = 1, fisher = True, bias = True, nan_policy = 'propagate')
    tmp4_ds = skew(feature_CST_ds, axis = 1, bias=True)    
    feature = np.vstack((tmp1,tmp2,tmp3,tmp4,
                         #tmp1-tmp11,tmp2-tmp22,tmp3-tmp33,tmp4-tmp44,
                         #tmp1_n,tmp2_n,tmp3_n,tmp4_n,
                         tmp1_ds, tmp2_ds,tmp3_ds,tmp4_ds
                        )).T
    return featureO

In [6]:
def imputation_feature_extraction(idx=subjN):
    from sklearn.impute import SimpleImputer
    from sklearn.impute import KNNImputer
    imp_median = KNNImputer(n_neighbors=10)
    m0 = np.argwhere(y==0)
    m1 = np.argwhere(y==1)

    #imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
    strm_median_P = imp_median.fit_transform(strm_median_P_n[idx,:])
    strm_median_H = imp_median.fit_transform(strm_median_H_n[idx,:])
    strm_median_P_adc = imp_median.fit_transform(strm_median_P_adc_n[idx,:])
    strm_median_H_adc = imp_median.fit_transform(strm_median_H_adc_n[idx,:])
    strm_median_P_rd = imp_median.fit_transform(strm_median_P_rd_n[idx,:])
    strm_median_H_rd = imp_median.fit_transform(strm_median_H_rd_n[idx,:])
    strm_median_P_ad = imp_median.fit_transform(strm_median_P_ad_n[idx,:])
    strm_median_H_ad = imp_median.fit_transform(strm_median_H_ad_n[idx,:])
    strm_mean_P_fd = imp_median.fit_transform(strm_mean_P_fd_n[idx,:])
    strm_mean_H_fd = imp_median.fit_transform(strm_mean_H_fd_n[idx,:])
    
    feature_fa = extract_stat_per_hemis(strm_median_P,strm_median_H)
    feature_adc = extract_stat_per_hemis(strm_median_P_adc,strm_median_H_adc)
    feature_rd = extract_stat_per_hemis(strm_median_P_rd,strm_median_H_rd)
    feature_ad = extract_stat_per_hemis(strm_median_P_ad,strm_median_H_ad)
    feature_fd = extract_stat_per_hemis(strm_mean_P_fd,strm_mean_H_fd)
    
    feature_CST = np.hstack((feature_fa,feature_adc))  # remove feature_adc
    X = feature_CST
    return X  

In [7]:
y = motor_status_preop
sub_all = np.arange(0,116,1).tolist()
X1 = imputation_feature_extraction(idx=sub_all)
n_features = 5
n_repeats = 500

In [61]:
##### try to implement RFE by myself
from numpy import mean
from numpy import std
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, f1_score, auc, confusion_matrix,plot_roc_curve
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.feature_selection import RFE, RFECV
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler, SVMSMOTE
from sklearn.model_selection import GridSearchCV

outer_results = list()
outer_results2 = list()
outer_results3 = list()
specificity = list()

tns = list()
fps = list()
tps = list()
fns = list()
scaler = MinMaxScaler()
scaler1 = StandardScaler()

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(20,20))

cv_outer = RepeatedStratifiedKFold(n_splits=10, n_repeats=1000, random_state=1)
k = 0
for i,(train_ix, test_ix) in enumerate(cv_outer.split(X1,y)):
    X1_train, X1_test = X1[train_ix, :], X1[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]

    
    X1_train = scaler1.fit_transform(X1_train)
    X1_test = scaler1.transform(X1_test)

    model1 = svm.SVC(kernel='linear',class_weight='balanced',gamma='scale', C=1) #3 set C = 0.5 gamma='scale',
    
    sfs1 = RFE(estimator=model1, step=1, n_features_to_select=n_features)

    
    sfs1 = sfs1.fit(X1_train, y_train)

    x_train_sfs = sfs1.transform(X1_train)
    x_test_sfs = sfs1.transform(X1_test)

    x_train_sfs = scaler1.fit_transform(x_train_sfs)
    x_test_sfs = scaler1.transform(x_test_sfs)
    
    cv_inner = RepeatedStratifiedKFold(n_splits=5, random_state=1)
    model2 = svm.SVC(kernel='linear',class_weight='balanced', gamma='scale', C = 1, probability=True)
    space = dict()
    space['C'] =np.arange(0.1,2,0.1).tolist()#[0.1,0.2,0.3,1,2,3,4]#np.arange(2**(-10),0.1,2*(10)).tolist()
    
    #space['gamma'] = ['auto','scale']
    #space['max_features'] = [2, 4, 6]
    # define search
    search = GridSearchCV(model2,space, cv=cv_inner, refit=True)
    result = search.fit(x_train_sfs, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(x_test_sfs)
    
    model2.fit(x_train_sfs, y_train)

    prediction = model2.predict_proba(x_test_sfs)
    prediction = best_model.predict_proba(x_test_sfs)
    fpr, tpr, t = roc_curve(y_test.reshape(-1,1), prediction[:, 1])
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    yhat = model2.predict(x_test_sfs)
    #yhat = best_model.predict(x_test_sfs)
    acc = accuracy_score(y_test, yhat)    
    acc2 = recall_score(y_test, yhat)
    acc3 = precision_score(y_test, yhat)
    tn, fp, fn, tp = confusion_matrix(y_test, yhat).ravel()
    specificity.append(tn/(tn+fp)) 
    tns.append(tn), fps.append(fp), fns.append(fn), tps.append(tp)

    outer_results.append(acc)
    outer_results2.append(acc2)
    outer_results3.append(acc3)
    # report progress     
    # print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    print('>acc=%.3f acc2=%.3f acc3=%.3f' % (acc,acc2,acc3))
    #print(test_ix)
    #print(test_ix[np.argwhere(y_test!=yhat)])
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))
print('Recall: %.3f (%.3f)' % (mean(outer_results2), std(outer_results2)))
print('Precision: %.3f (%.3f)' % (mean(outer_results3), std(outer_results3)))

plt.plot([0,1],[0,1],linestyle = '--',lw = 4,color = 'r',alpha=0.8)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std = np.std(aucs)

plt.plot(mean_fpr, mean_tpr, color='blue',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std),lw=4, alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.2,label=r'$\pm$ 1 std. dev.')

plt.xlabel('False Positive Rate',fontsize = 20)
plt.ylabel('True Positive Rate',fontsize = 20)
plt.title('Cross-Validation ROC of SVM',fontsize = 20)
plt.legend(loc="lower right", prop={'size': 20})
plt.xlim([-0.01, 1.01]) 
plt.ylim([-0.01, 1.01])   
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 2
plt.show()
plt.savefig('final.png')


>acc=0.667 acc2=0.500 acc3=0.500
>acc=0.583 acc2=0.400 acc3=0.500
>acc=0.500 acc2=0.600 acc3=0.429
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.750 acc2=1.000 acc3=0.625
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.818 acc2=1.000 acc3=0.667
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.636 acc2=0.500 acc3=0.500
>acc=0.833 acc2=1.000 acc3=0.667
>acc=0.583 acc2=0.800 acc3=0.500
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.750 acc2=0.400 acc3=1.000
>acc=0.583 acc2=0.800 acc3=0.500
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.455 acc2=0.500 acc3=0.333
>acc=0.909 acc2=1.000 acc3=0.800
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.667 acc2=0.750 acc3=0.500
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.583 acc2=0.800 acc3=0.500
>acc=0.833 acc2=0.800 acc3=0.800
>acc=0.667 acc2=1.000 acc3=0.556
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.727 acc2=0.500 acc3=0.667
>acc=0.909 acc2=1.000 acc3=0.800
>acc=0.636 acc2=0.500 acc3=0.500
>acc=0.667

>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.455 acc2=0.500 acc3=0.333
>acc=0.500 acc2=0.500 acc3=0.333
>acc=1.000 acc2=1.000 acc3=1.000
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.583 acc2=0.600 acc3=0.500
>acc=0.750 acc2=1.000 acc3=0.625
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.909 acc2=1.000 acc3=0.800
>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.455 acc2=0.250 acc3=0.250
>acc=0.667 acc2=0.750 acc3=0.500
>acc=0.583 acc2=0.400 acc3=0.500
>acc=0.833 acc2=1.000 acc3=0.714
>acc=0.917 acc2=1.000 acc3=0.833
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.833 acc2=1.000 acc3=0.714
>acc=0.545 acc2=0.750 acc3=0.429
>acc=0.545 acc2=0.250 acc3=0.333
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.909 acc2=0.750 acc3=1.000
>acc=0.750 acc2=0.500 acc3=0.667
>acc=0.833 acc2=0.800 acc3=0.800
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.583 acc2=0.400 acc3=0.500
>acc=0.500 acc2=0.400 acc3=0.400
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.364 acc2=0.500 acc3=0.286
>acc=0.545

>acc=0.909 acc2=1.000 acc3=0.800
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.583 acc2=0.250 acc3=0.333
>acc=0.750 acc2=1.000 acc3=0.625
>acc=0.667 acc2=0.400 acc3=0.667
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.583 acc2=0.600 acc3=0.500
>acc=0.750 acc2=1.000 acc3=0.625
>acc=0.636 acc2=1.000 acc3=0.500
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.545 acc2=1.000 acc3=0.444
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.583 acc2=0.250 acc3=0.333
>acc=0.500 acc2=0.600 acc3=0.429
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.833 acc2=0.800 acc3=0.800
>acc=0.417 acc2=0.400 acc3=0.333
>acc=0.750 acc2=1.000 acc3=0.625
>acc=0.909 acc2=1.000 acc3=0.800
>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.818 acc2=1.000 acc3=0.667
>acc=0.750 acc2=0.750 acc3=0.600
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.750 acc2=0.600 acc3=0.750
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.818 acc2=1.000 acc3=0.667
>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.818

>acc=0.583 acc2=0.600 acc3=0.500
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.909 acc2=1.000 acc3=0.800
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.667 acc2=0.750 acc3=0.500
>acc=0.583 acc2=0.400 acc3=0.500
>acc=0.500 acc2=0.800 acc3=0.444
>acc=0.833 acc2=0.800 acc3=0.800
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.833 acc2=0.600 acc3=1.000
>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.636 acc2=1.000 acc3=0.500
>acc=0.818 acc2=0.500 acc3=1.000
>acc=0.818 acc2=1.000 acc3=0.667
>acc=0.667 acc2=0.500 acc3=0.500
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.583 acc2=0.200 acc3=0.500
>acc=0.833 acc2=1.000 acc3=0.714
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.833 acc2=0.800 acc3=0.800
>acc=0.545 acc2=0.750 acc3=0.429
>acc=0.727 acc2=1.000 acc3=0.571
>acc=0.636 acc2=1.000 acc3=0.500
>acc=0.636 acc2=0.500 acc3=0.500
>acc=0.750 acc2=0.750 acc3=0.600
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.500 acc2=0.600 acc3=0.429
>acc=0.833 acc2=0.800 acc3=0.800
>acc=0.750

>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.727 acc2=1.000 acc3=0.571
>acc=0.167 acc2=0.500 acc3=0.200
>acc=0.583 acc2=0.600 acc3=0.500
>acc=0.583 acc2=0.600 acc3=0.500
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.750 acc2=0.600 acc3=0.750
>acc=0.833 acc2=1.000 acc3=0.714
>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.727 acc2=0.500 acc3=0.667
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.583 acc2=0.750 acc3=0.429
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.636 acc2=0.500 acc3=0.500
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.750 acc2=1.000 acc3=0.571
>acc=0.833 acc2=1.000 acc3=0.714
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.583 acc2=0.800 acc3=0.500
>acc=0.500 acc2=0.400 acc3=0.400
>acc=0.750 acc2=0.600 acc3=0.750
>acc=0.364 acc2=0.000 acc3=0.000
>acc=0.909

...

>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.636 acc2=0.500 acc3=0.500
>acc=0.455 acc2=0.250 acc3=0.250
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.750 acc2=0.750 acc3=0.600
>acc=0.750 acc2=0.600 acc3=0.750
>acc=0.833 acc2=1.000 acc3=0.714
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.583 acc2=0.600 acc3=0.500
>acc=0.833 acc2=0.800 acc3=0.800
>acc=0.818 acc2=0.500 acc3=1.000
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.455 acc2=0.750 acc3=0.375
>acc=0.636 acc2=0.750 acc3=0.500
>acc=0.667 acc2=0.750 acc3=0.500
>acc=0.500 acc2=0.400 acc3=0.400
>acc=0.833 acc2=0.800 acc3=0.800
>acc=0.583 acc2=0.800 acc3=0.500
>acc=0.833 acc2=1.000 acc3=0.714
>acc=0.500 acc2=0.400 acc3=0.400
>acc=0.818 acc2=1.000 acc3=0.667
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.636 acc2=0.500 acc3=0.500
>acc=0.818 acc2=1.000 acc3=0.667
>acc=0.750 acc2=0.750 acc3=0.600
>acc=0.750 acc2=0.800 acc3=0.667
>acc=0.750 acc2=0.600 acc3=0.750
>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.583 acc2=0.800 acc3=0.500
>acc=0.583

>acc=0.667 acc2=0.800 acc3=0.571
>acc=0.636 acc2=0.250 acc3=0.500
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.636 acc2=0.500 acc3=0.500
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.750 acc2=0.750 acc3=0.600
>acc=0.917 acc2=0.800 acc3=1.000
>acc=0.583 acc2=0.800 acc3=0.500
>acc=0.583 acc2=1.000 acc3=0.500
>acc=0.500 acc2=0.200 acc3=0.333
>acc=0.667 acc2=0.600 acc3=0.600
>acc=0.727 acc2=1.000 acc3=0.571
>acc=0.818 acc2=0.750 acc3=0.750
>acc=0.636 acc2=0.250 acc3=0.500
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.917 acc2=1.000 acc3=0.800
>acc=0.917 acc2=1.000 acc3=0.833
>acc=0.500 acc2=0.600 acc3=0.429
>acc=0.583 acc2=0.600 acc3=0.500
>acc=0.417 acc2=0.200 acc3=0.250
>acc=0.750 acc2=1.000 acc3=0.625
>acc=0.727 acc2=0.750 acc3=0.600
>acc=0.636 acc2=0.500 acc3=0.500
>acc=0.545 acc2=0.500 acc3=0.400
>acc=0.818 acc2=0.750 acc3=0.750
Accuracy: 0.688 (0.130)
Recall: 0.705 (0.213)
Precision: 0.598 (0.172)


ValueError: Image size of 1176x290130 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1440x1440 with 1 Axes>

<Figure size 432x288 with 0 Axes>